# Sales Analysis

## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import matplotlib.pyplot as plt

# global font family
plt.rcParams['font.family'] = 'Arial'

## Reading CSVs

In [2]:
location = 'D:\ANISH\Portfolio\Python Projects\Data Analysis Project\Dataset/*.csv'
for file in glob.glob(location):
    print(file)

In [3]:
df = pd.read_csv('D:\ANISH\Portfolio\Python Projects\Data Analysis Project\Dataset\Sales_January_2019.csv')
df.head()

## Function to combine CSV files

In [4]:
def combinedata(x):
        dfs = []
        for file in glob.glob(x):
            dfs.append(pd.read_csv(file))
            combineddfs = pd.concat(dfs,ignore_index=True)
        return combineddfs

filepath = 'D:\ANISH\Portfolio\Python Projects\Data Analysis Project\Dataset/*'

maindf = combinedata(filepath)
maindf.shape

## Final combined dataframe

In [5]:
maindf

# Cleaning dataframe

### Removing NULLS

In [6]:
maindf.isna().sum()
nan = maindf[maindf.isna().any(axis=1)]
maindf = maindf.dropna(how='all')
maindf.isna().sum()

### Removing Column names from rows

In [7]:
maindf[maindf['Order ID'] == "Order ID"]
maindf[maindf['Order ID'] == "Order ID"].count()

In [8]:
maindf = maindf.drop(maindf[maindf['Order ID'] == 'Order ID'].index)
maindf[maindf['Order ID'] == "Order ID"].count()

## Changing Datatypes to increase speed

In [9]:
maindf.info()
maindf['Quantity Ordered'] = pd.to_numeric(maindf['Quantity Ordered'])
maindf['Price Each'] = pd.to_numeric(maindf['Price Each'])
maindf['Order ID'] = pd.to_numeric(maindf['Order ID'])
maindf['Order Date'] = pd.to_datetime(maindf['Order Date'])
maindf.info()

## adding columns

In [10]:
#Rename Order Date to Order Datetime
df.rename(columns={'Order Date': 'Order DateTime'}, inplace=True)

#Date
maindf['Date'] = maindf['Order Date'].dt.date

# Month column
maindf['Month'] = maindf['Order Date'].dt.month

# weekday
maindf['Weekday'] = maindf['Order Date'].dt.weekday

# Day column
maindf['MonthDay'] = maindf['Order Date'].dt.day

# Hour column
maindf['Hour'] = maindf['Order Date'].dt.hour

# Minute column
maindf['Minute'] = maindf['Order Date'].dt.minute

In [11]:
# Sales Column
maindf['Sales'] = maindf['Quantity Ordered'] * maindf['Price Each']

In [12]:
# City State column
# City State
def get_city_state(x):
    return f"{x.split(', ')[1]} {x.split(', ')[2].split(' ')[0]}"

maindf['City State'] = maindf['Purchase Address'].apply(lambda x: get_city_state(x))

In [13]:
maindf.info()

In [14]:
maindf

### 1. What product sold the most? 

In [15]:
best_product = maindf.groupby('Product')
products = [product for product, maindf in best_product]
quantity_sold = best_product['Quantity Ordered'].sum()

plt.barh(products, quantity_sold,color=['#D0693A','#3B8EA5'])
plt.xlabel('Quantity Sold')
plt.ylabel('Product')
plt.title('Quantity Sold for Each Product')
plt.show()

### 3. Best time to display advertisements in the day.

In [16]:
# heatmap
best_hours = maindf.groupby('Hour') 
hour = [hour for hour, maindf in best_hours]
count_hour = best_hours['Hour'].count()
plt.plot(hour,count_hour, marker='o',markersize = 4)
plt.xticks(hour)
plt.xlabel("Hour of the day")
plt.ylabel("Orders")
plt.grid(True, linestyle='--', alpha=0.8)
plt.show()

### 5. Best month for sales

In [17]:
best_month = maindf.groupby('Month').sum()
months = range(1,13)
plt.bar(months,best_month['Sales'])
plt.xticks(months)
plt.xlabel('Month')
plt.ylabel('Sales ($)')
plt.show()

### What city sold the most products?

In [18]:
best_city = maindf.groupby('City State')
city = [city for city, maindf in best_city]
city_sales = best_city['Sales'].sum()

plt.bar(city, city_sales,color=['#D0693A','#3B8EA5'])
plt.xticks(city, rotation = 65)
plt.xlabel('City')
plt.ylabel('Sales ($)')
plt.title('Sales')
plt.show()

### Sales Trend

In [19]:
sns.relplot(
    data=maindf, 
    x="Month", y="Sales", hue="City State", 
    height=5, aspect=2,
    kind="line",
    errorbar=None
)

In [20]:
maindf['Weekend'] = maindf['MonthDay'].isin([5, 6])
sales_by_weekend = maindf.groupby('Weekend')['Sales'].sum()
# Create a bar plot to visualize the sales variation between weekdays and weekends
sales_by_weekend.plot(kind='bar', color=['#3B8EA5', '#D0693A'], alpha=0.9)

# Customize the plot labels and title
plt.xlabel('Week')
plt.ylabel('Sales')
plt.title('Sales Variation: Weekdays vs. Weekends')

# Customize the x-axis tick labels
plt.xticks([0, 1], ['Weekday', 'Weekend'],rotation = 'horizontal')

# Display the plot
plt.show()

In [21]:
best_city = maindf.groupby('City State')
city = [city for city, maindf in best_city]
city_sales = best_city['Sales'].sum()

plt.bar(city, city_sales,color=['#D0693A','#3B8EA5'])
plt.xticks(city, rotation = 65)
plt.xlabel('City')
plt.ylabel('Sales ($)')
plt.title('Sales')
plt.show()

In [22]:
maindf.info()
print(maindf.columns)

In [23]:
maindf

In [24]:
barrace = maindf.pivot_table(index=['Date'],columns=['City State'], values='Sales',aggfunc="sum",fill_value=0)
barrace

In [25]:
barrace.info()

In [ ]:
import bar_chart_race as bcr
bcr.bar_chart_race(
    df=barrace,
    filename='Salesbycity.mp4',  # Save as a video file
    orientation='h',  # Horizontal bars
    sort='desc',  # Sort bars by descending order
    n_bars=10,  # Number of bars to display
    fixed_order=False,  # Allow bars to change order
    fixed_max=True,  # Fixed maximum value for the bars
    steps_per_period=10,  # Number of frames per period
    interpolate_period=False,  # No interpolation between periods
    label_bars=True,  # Show labels on bars
    bar_size=0.95,  # Bar width
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},  # Position of the period label
    period_fmt='%B %d, %Y',  # Period format
    period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                      's': f'Total Sales: {v.nlargest(6).sum():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Arial'},  # Show total sales
    perpendicular_bar_func='median',  # Bars' position is calculated based on median
    period_length=500,  # Duration of each period (in milliseconds)
    figsize=(5, 3),  # Size of the output figure
    dpi=144,  # Resolution of the image
    cmap='dark12',  # Color map for the bars
    title='Sales by City',  # Title of the chart
    title_size=14,  # Size of the title
    bar_label_size=7,  # Size of the bar labels
    tick_label_size=7,  # Size of the tick labels
    shared_fontdict={'family': 'Arial', 'color': '.1'},  # Shared font for the labels
    scale='linear',  # Linear scale for the chart
    writer=None,  # Writer for the video (set to None to not use a writer)
    fig=None,  # No specific figure to use
    bar_kwargs={'alpha': .7},  # Transparency for the bars
    filter_column_colors=False  # Don't filter colors of the columns
)